## 2.1 Introduction to Introduction to Big Data and Data Engineering

### Initial Setup for MongoDB Access

In [1]:
# Ensure pymongo is installed
# ------------------------------------------------------------------------------
!python -m pip install "pymongo"

In [ ]:
# Connecting to MongoDB Atlas

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi


uri = "mongodb+srv://spidey_dbuser:@spideycluster.7fu5sbe.mongodb.net/?appName=SpideyCluster"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [4]:
import pymongo

In [7]:
client.list_database_names()

['sample_mflix', 'admin', 'local']

In [ ]:
# Connect to the sample_mflix database
db = client.sample_mflix

In [12]:
movies = db.movies

In [13]:
db.list_collection_names()

['users', 'sessions', 'embedded_movies', 'comments', 'movies', 'theaters']

### Question 1 - From the `movies` collection, return the documents with the `plot` that starts with `"war"` in acending order of released date, print only title, plot and released fields. Limit the result to 5.


In [30]:
# Query 1: Find movies with plots starting with "war", sorted by release year, return title/plot/released, limit 5
result = movies.find(
    {'plot': {'$regex': '^War', '$options': 'i'}},
    {'title': 1, 'plot': 1, 'released': 1, '_id': 0}
).sort("released", pymongo.ASCENDING).limit(5)

for movie in result:
    print(f"Title: {movie['title']}")
    print(f"Plot: {movie['plot']}")
    print(f"Released: {movie['released']}")
    print(">--------------------------------------------------------------------------------------------<")

Title: Nausicaè of the Valley of the Wind
Plot: Warrior/pacifist Princess Nausicaè desperately struggles to prevent two warring nations from destroying themselves and their dying planet.
Released: 1984-03-11 00:00:00
>--------------------------------------------------------------------------------------------<
Title: Nausicaè of the Valley of the Wind
Plot: Warrior/pacifist Princess Nausicaè desperately struggles to prevent two warring nations from destroying themselves and their dying planet.
Released: 1984-03-11 00:00:00
>--------------------------------------------------------------------------------------------<
Title: Heaven and Earth
Plot: Warlords Kagetora and Takeda each wish to prevent the other from gaining hegemony in feudal Japan. The two samurai leaders pursue one another across the countryside, engaging in massive ...
Released: 1991-02-08 00:00:00
>--------------------------------------------------------------------------------------------<
Title: Under the Stars
Plot: Wa

### Question 2 - Group by `rated` and count the number of movies in each.

In [36]:
stage_group_rated = {
   "$group": {
         "_id": "$rated", # Group by the 'rated' field
         # Count the number of movies in the group:
         "movie_count": { "$sum": 1 }, 
   }
}

pipeline = [
   stage_group_rated,
]
results = movies.aggregate(pipeline)

# Loop through the 'rated-summary' documents:
for rated_summary in results:
   print(rated_summary)

{'_id': None, 'movie_count': 9894}
{'_id': 'APPROVED', 'movie_count': 709}
{'_id': 'M', 'movie_count': 37}
{'_id': 'Not Rated', 'movie_count': 1}
{'_id': 'TV-14', 'movie_count': 89}
{'_id': 'TV-PG', 'movie_count': 76}
{'_id': 'OPEN', 'movie_count': 1}
{'_id': 'TV-G', 'movie_count': 59}
{'_id': 'G', 'movie_count': 477}
{'_id': 'PASSED', 'movie_count': 181}
{'_id': 'TV-MA', 'movie_count': 60}
{'_id': 'PG-13', 'movie_count': 2321}
{'_id': 'PG', 'movie_count': 1852}
{'_id': 'GP', 'movie_count': 44}
{'_id': 'TV-Y7', 'movie_count': 3}
{'_id': 'Approved', 'movie_count': 5}
{'_id': 'R', 'movie_count': 5537}
{'_id': 'AO', 'movie_count': 3}


### Question 3 - Count the number of movies with 3 comments or more.

In [52]:
# Question 3: Count movies with 3 or more comments using $group with timeout handling
from pymongo.errors import ExecutionTimeout

stage_lookup_comments = {
    "$lookup": {
        "from": "comments",
        "localField": "_id",
        "foreignField": "movie_id",
        "as": "related_comments",
    }
}

stage_add_comment_count = {
    "$addFields": {
        "comment_count": {
            "$size": "$related_comments"
        }
    }
}

stage_match_3_or_more = {
    "$match": {
        "comment_count": {
            "$gte": 3
        }
    }
}

stage_group_count = {
    "$group": {
        "_id": None,
        "movies_with_3_or_more_comments": {"$sum": 1},
    }
}

pipeline = [
    stage_lookup_comments,
    stage_add_comment_count,
    stage_match_3_or_more,
    stage_group_count,
]

try:
    results = movies.aggregate(pipeline, maxTimeMS=600000)  # 10min timeout
    for result in results:
        print(result)
except ExecutionTimeout:
    print("Query timed out. Consider optimizing your aggregation pipeline.")
except Exception as e:
    print(f"Error: {e}")

Query timed out. Consider optimizing your aggregation pipeline.


#### Elegant solution from Kieron - - attached for my learning

In [ ]:
pipeline = [
    { "$match": { "num_mflix_comments": { "$gte": 3 } } },
    { "$group": { "_id": None, "movies_count": { "$sum": 1 } } }
]

result = movies.aggregate(pipeline).next()
print(f"Number of movies with 3 comments or more: {result['movies_count']}")

Number of movies with 3 comments or more: 385


#### Alternative solution from Darwin - attached for my learning

In [ ]:

stage_match_with_comments = {
   "$match": {
         "num_mflix_comments": {
            "$gte": 3
         }
   } 
}

stage_group_count = {
   "$group": {
         "_id": None,
         "movies_count": { "$sum": 1 }
   }
}

pipeline = [
   stage_match_with_comments,
   stage_group_count
]

results = movies.aggregate(pipeline)

for result in results:
   print(f"Number of movies with 3 comments or more: {result['movies_count']}")

Number of movies with 3 comments or more: 385


#### Another solution from Farida - attached for my learning

In [60]:
# Count movies with 3 or more comments
count = movies.count_documents({"num_mflix_comments": {"$gte": 3}})
print("Number of movies with 3 or more comments:", count)


Number of movies with 3 or more comments: 385


#### My solution for Q3 was worked in tandem with VS Code's AI - which timed out
#### Attached codes from Kieron, Darwin and Farida on Q3 for my learning

#### Including Thomas' solutions

In [15]:
### There are 2 solutions
### Method 1 uses only movies dataset.
### Sometimes due to server load, Method 2 may timeout at certain time of the day.
### Result is 385 for both method.

### Method 1 : Use only movies dataset, No timeout
# Match movie documents with at least 3 comment:
stage_match_with_comments = {
   "$match": {
         "num_mflix_comments": {
            "$gte": 3
         }
   } 
}

stage_count_movies = {
   "$count": "number_movies_with_3_comments_or_more"
} 


pipeline = [
   stage_match_with_comments,
   stage_count_movies
]
results = movies.aggregate(pipeline)

# Print Result:
for summary in results:
   print(summary)

{'number_movies_with_3_comments_or_more': 385}


In [16]:
### Alternative Solution:
### Method 2 : Use $lookup and $match method, may time out during certain time of the day. Result is the same.
# Look up related documents in the 'comments' collection:
stage_lookup_comments = {
   "$lookup": {
         "from": "comments", 
         "localField": "_id", 
         "foreignField": "movie_id", 
         "as": "related_comments",
   }
}

# Calculate the number of comments for each movie:
stage_add_comment_count = {
   "$addFields": {
         "comment_count": {
            "$size": "$related_comments"
         }
   } 
}

# Match movie documents with at least 3 comment:
stage_match_with_comments = {
   "$match": {
         "comment_count": {
            "$gte": 3
         }
   } 
}

# Count number of movies
stage_match_movies_count = {
   "$count": "movies_count" 
}


pipeline = [
   stage_lookup_comments,
   stage_add_comment_count,
   stage_match_with_comments,
   stage_match_movies_count
]

results = movies.aggregate(pipeline)
for summary in results:   
    print(summary)    

ExecutionTimeout: PlanExecutor error during aggregation :: caused by :: operation exceeded time limit, full error: {'ok': 0.0, 'errmsg': 'PlanExecutor error during aggregation :: caused by :: operation exceeded time limit', 'code': 50, 'codeName': 'MaxTimeMSExpired', '$clusterTime': {'clusterTime': Timestamp(1763101836, 1), 'signature': {'hash': b'-X\xe6\xa1\xf7O\x88>mY\n\xed\x12\x07}~\xbc\xeb^*', 'keyId': 7512907272637382750}}, 'operationTime': Timestamp(1763101836, 1)}